In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder


In [ ]:
loaded_model = tf.keras.models.load_model("model/skimlit_tribrid_model")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/SC'
/content/drive/My Drive/SC


In [ ]:
def classify(abstract):

  target={0:'BACKGROUND', 1:'CONCLUSIONS', 2:'METHODS', 3:'OBJECTIVE', 4:'RESULTS'}

  # Make function to split sentences into characters
  def split_chars(text):
    return " ".join(list(text))


  # Create sentencizer - Source: https://spacy.io/usage/linguistic-features#sbd 
  from spacy.lang.en import English
  nlp = English() # setup English sentence parser
  #sentencizer = nlp.create_pipe("sentencizer") # create sentence splitting pipeline object
  nlp.add_pipe(nlp.create_pipe('sentencizer')) # add sentence splitting pipeline object to sentence parser
  doc = nlp(abstract) # create "doc" of parsed sequences, change index for a different abstract
  abstract_lines = [str(sent) for sent in list(doc.sents)] # return detected sentences from doc in string type (not spaCy token type)
  abstract_lines


  total_lines_in_sample = len(abstract_lines)
  # Go through each line in abstract and create a list of dictionaries containing features for each line
  sample_lines = []
  for i, line in enumerate(abstract_lines):
    sample_dict = {}
    sample_dict["text"] = str(line)
    sample_dict["line_number"] = i
    sample_dict["total_lines"] = total_lines_in_sample - 1
    sample_lines.append(sample_dict)
  sample_lines


  # Get all line_number values from sample abstract
  test_abstract_line_numbers = [line["line_number"] for line in sample_lines]
  # One-hot encode to same depth as training data, so model accepts right input shape
  test_abstract_line_numbers_one_hot = tf.one_hot(test_abstract_line_numbers, depth=15) 
  test_abstract_line_numbers_one_hot


  # Get all total_lines values from sample abstract
  test_abstract_total_lines = [line["total_lines"] for line in sample_lines]
  # One-hot encode to same depth as training data, so model accepts right input shape
  test_abstract_total_lines_one_hot = tf.one_hot(test_abstract_total_lines, depth=20)
  test_abstract_total_lines_one_hot

  # Split abstract lines into characters
  abstract_chars = [split_chars(sentence) for sentence in abstract_lines]
  abstract_chars

  # Make predictions on sample abstract features
  test_abstract_pred_probs = loaded_model.predict(x=(test_abstract_line_numbers_one_hot,
                                                   test_abstract_total_lines_one_hot,
                                                   tf.constant(abstract_lines),
                                                   tf.constant(abstract_chars)))
  test_abstract_pred_probs

  # Turn prediction probabilities into prediction classes
  test_abstract_preds = tf.argmax(test_abstract_pred_probs, axis=1)
  test_abstract_preds=test_abstract_preds.numpy()
  test_abstract_preds

  # Turn prediction class integers into string class names
  test_abstract_pred_classes = [target[i] for i in test_abstract_preds]
  test_abstract_pred_classes

  s=""
  for i in range (total_lines_in_sample):
    s=s+test_abstract_pred_classes[i]+" : "+abstract_lines[i]+"\n"
    
  return s